In [ ]:
import os
import re
import glob
import pandas as pd
import itertools

def get_group(filename):
    pattern = r'grid_\d+x\d+'
    match = re.search(pattern, filename)
    if match:
        extracted = match.group()
    else:
        extracted = 'orig'
    return extracted

def get_patch(filename):
    pattern = r'row\d+_col\d+'
    match = re.search(pattern, filename)
    if match:
        extracted = match.group()
    else:
        extracted = 'orig'
    return extracted


base_dir = '/home/srikapan/workspace/research/multiblock/VLMEvalKit/outputs'
model_name = 'Phi-3.5-Vision' # llava_hf_v1.5_7b, InternVL2-1B, Idefics3-8B-Llama3, Phi-3.5-Vision
dataset_name = 'RealWorldQA'
file_paths = glob.glob(f'{base_dir}/{model_name}/{model_name}_{dataset_name}_*openai_result.xlsx')
groups = {}
for i in file_paths:
    group = get_group(os.path.basename(i))
    if group not in groups:
        groups[group] = [os.path.basename(i)]
    else:
        groups[group].append(os.path.basename(i))

all_combinations = []
for r in range(1, len(groups.keys()) + 1):
    all_combinations.extend(itertools.combinations(groups.keys(), r))

final_dict = {}
for gp_element in all_combinations:
    all_excel = {}
    for indi_element in gp_element:
        for i in groups[indi_element]:
            df = pd.read_excel(f'{base_dir}/{model_name}/{i}')
            hit = df.hit.tolist()
            all_excel[os.path.basename(i).replace('.xlsx', '')] = hit
    result = [int(any(values)) for values in zip(*all_excel.values())]
    result = sum(result)/len(result)
    final_dict['-'.join(gp_element)] = result
df = pd.DataFrame(list(final_dict.items()), columns=['Configuration', 'Value'])
df.to_csv(f'{base_dir}/{model_name}_sumary.csv', index=False)